# **TP Titanic**

In [26]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import neighbors

In [27]:
# Opening CSV
data = pd.read_csv(r'../data/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
# Cleaning
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
# Dummies for Sex
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)
# Result
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


In [29]:
explicativeVariables = data.drop('Survived', axis=1)
explicativeVariables.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,3,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,0
2,3,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,0
4,3,35.0,0,0,8.0500,1


In [30]:
targetVariable = data[['Survived']]
targetVariable.head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [31]:
# Function for graphing
def graph(x, y, survived, xLabel, yLabel):
    plt.scatter(x, y, c=np.array(survived))
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.colorbar()
    plt.show()

In [32]:
def graphRelationBetweenTwoVariables(explicativeVariables, targetVariable):
    for i in range(0, explicativeVariables.columns.size - 1):
        for j in range(i + 1, explicativeVariables.columns.size):
            columnI = explicativeVariables.columns[i]
            columnJ = explicativeVariables.columns[j]
            graph(explicativeVariables[columnI], explicativeVariables[columnJ], targetVariable, columnI, columnJ)

In [33]:
#graphRelationBetweenTwoVariables(explicativeVariables, targetVariable)

# _**Exprópiese**_

In [36]:
def createLogisticModel(x, y):
    if 'const' in x.columns:
        x = x.drop('const', axis=1)

    return LogisticRegression().fit(x, y)

In [ ]:
def simpleTreeModel(X, y):
    if 'const' in X.columns:
        Xp = X.drop('const', axis=1)
    else:
        Xp = X
    
    model = tree.DecisionTreeClassifier(min_samples_leaf=5).fit(Xp, y)

    return model

In [ ]:
def randomForestModel(X, y):
    if 'const' in X.columns:
        Xp = X.drop('const', axis=1)
    else:
        Xp = X
    
    model = RandomForestClassifier(min_samples_leaf=5, n_estimators=100).fit(Xp, y)

    return model

In [ ]:
def calculateAccuracy(y_real, y_pred):
    

In [ ]:
# VER DE ADAPTAR EL KFOLD PARA ESTO
# def kFoldValidation(x, y, percentageOfTest=0.3):
#     mseArray = []
#     k = 1/percentageOfTest
#     for i in range(0, int(k)):
#         xTest = x[i * math.floor(len(x) / k) : (i + 1) * math.floor(len(x) / k)]
#         yTest = y[i * math.floor(len(y) / k) : (i + 1) * math.floor(len(y) / k)]
#         xTrain = pd.concat([x[0: i * math.floor(len(x) / k)], x[(i + 1) * math.floor(len(x) / k) :]])
#         yTrain = pd.concat([y[0: i * math.floor(len(y) / k)], y[(i + 1) * math.floor(len(y) / k) :]])
#         mseArray.append(mseForFold(xTrain.reset_index(), yTrain.reset_index(), xTest.reset_index(), yTest.reset_index()))
#     return meanMSE(mseArray)

# _**Expropiado**_